<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/bigtweet_ask_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-5.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-5.validation.csv",header=None)
valid.head()

,0,1
0,2,heel pain
1,2,heel pain
2,5,increase my heart rate
3,5,tachycardia
4,8,severe arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-5.test.csv",header=None)
test.head()

,0,1
0,0,fear
1,0,scared to death
2,2,heel pain
3,2,heels hurt
4,2,pain in my right heel


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'bigtweet_ask.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'lipitor',
 'in',
 'of',
 'muscle',
 'to',
 'the',
 'severe',
 'and',
 'my',
 'myalgia',
 'rt',
 'fatigue',
 'a',
 'lower',
 'loss',
 'cramp',
 'limb',
 'weakness',
 'for',
 'disease',
 'stomach',
 'back',
 'depression',
 '-',
 'on',
 'cramps',
 'is',
 'upper',
 'gas',
 'memory',
 'joint',
 'you',
 'leg',
 'legs',
 'pains',
 'i',
 'with',
 'headache',
 'unable',
 'shoulder',
 'arthralgia',
 'knee',
 'neck',
 'foot',
 'excessive',
 'aches',
 'amp',
 'hip',
 'gastrointestinal',
 'not',
 'numbness',
 'all',
 'insomnia',
 'arthritis',
 'like',
 'arthrotec',
 '/',
 'bleeding',
 'nausea',
 'gluten',
 'asthenia',
 'tired',
 'walk',
 'feeling',
 'abdominal',
 'dizziness',
 'walking',
 'heart',
 'problems',
 'feet',
 'difficulty',
 'swelling',
 'chest',
 'muscles',
 'hair',
 'zocor',
 'backache',
 'ache',
 'burning',
 'at',
 'extreme',
 'hand',
 'could',
 'vision',
 'stiffness',
 'it',
 'your',
 

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.234494,2.753880,0.550034,1:06:17


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.846207,2.700773,0.555682,1:07:31


In [25]:
learn.save_encoder('healthnews_ask_fold5_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('healthnews_ask_fold5_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.088788,3.725546,0.338442,04:10


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.138841,3.257210,0.434227,04:25


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.453557,2.771765,0.513410,10:01


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.069309,2.375957,0.560664,09:27
1,1.806106,2.217885,0.587484,10:09


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.709949,2.038651,0.624521,09:08
1,1.391485,1.910816,0.657727,09:41


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.288543,1.776322,0.676884,09:25
1,1.124113,1.673305,0.697318,09:58


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.112898,1.595256,0.702427,10:03
1,0.966454,1.514425,0.729246,10:48


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.890213,1.437631,0.742018,09:46
1,0.778386,1.387585,0.749681,10:09


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.737784,1.388751,0.747126,09:47
1,0.738381,1.341987,0.748404,09:36
2,0.594983,1.314710,0.761175,09:34
3,0.504734,1.279004,0.766284,09:13


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.534453,1.299568,0.753512,10:39
1,0.559216,1.300130,0.773946,09:48
2,0.446559,1.244140,0.773946,10:13
3,0.413912,1.233074,0.780332,10:37


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.390579,1.277708,0.782886,10:10
1,0.430480,1.222498,0.789272,09:38
2,0.370636,1.190568,0.798212,10:05
3,0.329711,1.196379,0.807152,10:35


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.344719,1.188388,0.799489,09:50
1,0.391513,1.248484,0.795658,09:24
2,0.325344,1.185299,0.814815,09:47
3,0.276530,1.223316,0.810983,10:36


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.297158,1.236156,0.802043,10:17
1,0.314499,1.211895,0.808429,10:53
2,0.291970,1.174141,0.813538,09:36
3,0.233360,1.212694,0.817369,09:54


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.270989,1.266066,0.817369,10:24
1,0.301775,1.246006,0.810983,09:56
2,0.244526,1.187086,0.814815,10:06
3,0.221298,1.221351,0.818646,09:54


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.224239,1.261072,0.818646,09:24
1,0.238754,1.192037,0.817369,10:06
2,0.233944,1.213533,0.823755,09:36
3,0.220510,1.212517,0.825032,10:09


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.221405,1.193887,0.814815,10:07
1,0.213425,1.235739,0.818646,10:20


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.234647,1.283020,0.807152,10:53
1,0.227734,1.222953,0.822478,10:39


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.189875,1.286569,0.813538,09:22


In [44]:
# save the best model

learn.save_encoder('healthnews_ask_mixed_fold5')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[0, 833, 2, 182, 111, 8, 8, 8, 100, 16, 16, 16, 19, 27, 536, 27, 816, 34, 34, 197, 37, 37, 37, 37, 37, 37, 859, 48, 544, 411, 49, 49, 49, 100, 53, 53, 53, 58, 544, 973, 832, 71, 71, 71, 71, 73, 209, 864, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 228, 80, 80, 220, 83, 733, 416, 92, 94, 94, 94, 94, 94, 94, 95, 96, 578, 96, 98, 100, 100, 100, 100, 100, 100, 100, 100, 100, 103, 104, 287, 107, 107, 108, 108, 108, 111, 730, 465, 111, 111, 111, 111, 289, 111, 26, 119, 163, 836, 734, 734, 125, 920, 30, 125, 125, 125, 125, 125, 135, 26, 137, 137, 344, 148, 148, 151, 157, 865, 375, 163, 166, 166, 166, 171, 172, 172, 289, 172, 172, 172, 172, 172, 172, 172, 173, 173, 214, 860, 26, 178, 181, 181, 182, 182, 183, 183, 183, 183, 183, 183, 183, 183, 184, 187, 187, 190, 190, 190, 190, 309, 462, 190, 190, 190, 190, 190, 190, 855, 193, 810, 197, 197, 197, 197, 197, 201, 202, 205, 209, 210, 210

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

668
0.7713625866050808
